In [1]:
from GCForest import gcForest
import pandas as pd
import numpy as np
import pickle 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")
#用来计算程序运行时间
import datetime
starttime = datetime.datetime.now()

In [2]:
#读取32个通道的EEG数据，每个通道包含32×40=1280个信号样本（人次×每人次40实验）
#每个样本向量大小为8064点（63s*128Hz）
for eegCH in range(1,33,1):
    file_path = "./dump_file/CH{}_df_EEG_x".format(eegCH)
    locals()['CH{}_df_EEG_x'.format(eegCH)] = pickle.load(open(file_path,"rb"))
    file_path = "./dump_file/CH{}eegfft_df".format(eegCH)
    locals()["CH{}eegfft_df".format(eegCH)] = pickle.load(open(file_path,"rb"))
    
#读取Y
all_df_y_2c = pickle.load(open("./dump_file/all_df_y_2c","rb"))
print("Y.shape:",all_df_y_2c.shape)

Y.shape: (1280, 1)


In [3]:
#CH1_df_EEG_x
#X = np.ones([1280,8064])
#输入需要是np.array格式

#CH1_df_EEG_x.fillna(value=0.0,inplace=True)
#X = CH1_df_EEG_x.values
CH1eegfft_df.fillna(value=0.0,inplace=True)
X = CH1eegfft_df.values

X = X.astype(np.int32)
min_max_scaler = MinMaxScaler()
X = min_max_scaler.fit_transform(X)

y = all_df_y_2c.values
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2)

print("X.shape:",X.shape)
print("y.shape:",y.shape)
print("X_tr.shape:",X_tr.shape)
print("X_te.shape:",X_te.shape)
print("y_tr.shape:",y_tr.shape)
print("y_te.shape:",y_te.shape)

X.shape: (1280, 8064)
y.shape: (1280, 1)
X_tr.shape: (1024, 8064)
X_te.shape: (256, 8064)
y_tr.shape: (1024, 1)
y_te.shape: (256, 1)


In [4]:
X

array([[0.1739764 , 0.54149201, 0.04717277, ..., 0.9174257 , 0.04717277,
        0.54149201],
       [0.17397675, 0.54149204, 0.04717278, ..., 0.91742572, 0.04717278,
        0.54149204],
       [0.17397683, 0.54149206, 0.04717278, ..., 0.91742573, 0.04717278,
        0.54149206],
       ...,
       [0.17370272, 0.54181181, 0.04701359, ..., 0.91760953, 0.04701359,
        0.54181181],
       [0.17390189, 0.54150023, 0.04706673, ..., 0.91772594, 0.04706673,
        0.54150023],
       [0.17356568, 0.54166338, 0.04718688, ..., 0.91758157, 0.04718688,
        0.54166338]])

In [5]:
#gcf = gcForest(shape_1X=[63,128], window=[4,8])
gcf = gcForest(shape_1X=8064, window=1024, stride=32,tolerance=0.0,n_cascadeRF=1, 
               min_samples_mgs=0.1, min_samples_cascade=0.1,n_jobs=19)
#gcf = gcForest(shape_1X=[63,128], window=[4,128], tolerance=0.0, min_samples_mgs=10, min_samples_cascade=7,n_jobs=-1)
gcf.fit(X_tr, y_tr)

Slicing Sequence...
Training MGS Random Forests...
Adding/Training Layer, n_layer=1
Layer validation accuracy = 0.6341463414634146
Adding/Training Layer, n_layer=2
Layer validation accuracy = 0.6341463414634146


In [6]:
pred_X = gcf.predict(X_te)

Slicing Sequence...


In [7]:
# evaluating accuracy
accuracy = accuracy_score(y_true=y_te, y_pred=pred_X)
print('gcForest accuracy : {}'.format(accuracy))

#用来计算程序运行时间
endtime = datetime.datetime.now()
print("程序运行时间:%.1fs"%(endtime - starttime).seconds)

gcForest accuracy : 0.64453125
程序运行时间:120.0s
